# Clasificación de Planes de Megaline

# Contenido <a id='back'></a>

* [Introducción](#intro)
* [Objetivos](#obj)
* [Etapa 1. Descripción de los datos](#data_review)
* [Etapa 2. Preprocesamiento de datos](#data_preprocessing)
* [Etapa 3. Desarrollo de modelos](#modelo)
* [Conclusiones](#end)

## Introducción <a id='intro'></a>
En este proyecto, exploraremos diferentes técnicas de aprendizaje automático para analizar el comportamiento de los clientes de Megaline, una compañía móvil, con el objetivo de recomendar uno de sus nuevos planes: Smart o Ultra. Actualmente, muchos clientes aún utilizan planes heredados, lo que la compañía desea cambiar mediante un modelo predictivo que pueda sugerir el plan más adecuado para cada usuario.

## Objetivos: <a id='obj'></a>
Desarrollar un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75.

## Etapas:
El proyecto consistirá en tres etapas:
 1. Descripción de los datos.
 2. Preprocesamiento de datos.
 3. Desarrollo de modelos.


## Etapa 1. Descripción de los datos <a id='data_review'></a>

Cada observación en el dataset contiene información del comportamiento mensual sobre un usuario. La información dada es la siguiente:

*сalls — número de llamadas,

*minutes — duración total de la llamada en minutos,

*messages — número de mensajes de texto,

*mb_used — Tráfico de Internet utilizado en MB,

*is_ultra — plan para el mes actual (Ultra - 1, Smart - 0).

## Etapa 2. Preprocesamiento de datos <a id='data_preprocessing)'>

In [13]:
#importamos librerias necesarias
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

## Exploracion Inicial de Datos

In [3]:
#cargamos el dataset
df = pd.read_csv("users_behavior.csv")
df.describe()

## Analisis de Datos

El dataset proporcionado contiene información detallada sobre el comportamiento mensual de los usuarios, incluyendo el número de llamadas realizadas, la duración total de las llamadas, la cantidad de mensajes de texto enviados y el tráfico de Internet utilizado, así como la etiqueta que indica si el usuario está actualmente suscrito al plan Ultra (1) o al plan Smart (0).

In [4]:
# Definimos el target y features
features= df.drop(['is_ultra'], axis = 1)
target= df['is_ultra']

In [17]:
# Segmentamos un conjunto de entrenamiento, de validacion y de prueba (60-20-20)
features_train, features_remaining, target_train, target_remaining = train_test_split(features, target, test_size=0.4, random_state=54321)
features_val, features_test, target_val, target_test = train_test_split(features_remaining, target_remaining, test_size=0.5, random_state=54321)

# Verificación de tamaños
print(f'Tamaño conjunto de entrenamiento: {len(features_train)}')
print(f'Tamaño conjunto de validacion: {len(features_val)}')
print(f'Tamaño conjunto de prueba: {len(features_test)}')

## Etapa 3. Desarrollo de modelos <a id='modelo'></a>

In [6]:
# Probamos el modelo de DecisionTreeClasifier
model1 = DecisionTreeClassifier(random_state=54321)
model1.fit(features_train, target_train)
prediccion1 = model1.predict(features_test)

# Calcular la exactitud del modelo
accuracy1 = accuracy_score(target_test, prediccion1)
print(f'Exactitud de Árbol de Decisión: {accuracy1}')


In [7]:
# Probamos el modelo RandomForestClassifier
model2 = RandomForestClassifier(random_state=54321)
model2.fit(features_train, target_train)
prediccion2 = model2.predict(features_test)

# Calcular la exactitud del modelo
accuracy2 = accuracy_score(target_test, prediccion2)
print(f'Exactitud de Bosque Aleatorio: {accuracy2}')

In [8]:
# Probamos el modelo de Regresión Logistica
model3 = LogisticRegression(random_state=54321)
model3.fit(features_train, target_train)
prediccion3 = model3.predict(features_test)

# Calcular la exactitud del modelo
accuracy3 = accuracy_score(target_test, prediccion3)
print(f'Exactitud de Regresión Lineal: {accuracy3}')

In [18]:
# Implementamos el loop para probar diferentes hiperparámetros en RandomForestClassifier
best_accuracy = 0
best_params = {}
for n_estimators in [50, 100, 200]:
    for max_depth in [None, 10, 20, 30]:
        for min_samples_split in [2, 5, 10]:
            model = RandomForestClassifier(
                n_estimators=n_estimators, 
                max_depth=max_depth, 
                min_samples_split=min_samples_split, 
                random_state=54321
            )
            model.fit(features_train, target_train)
            prediccion_val = model.predict(features_val)
            accuracy_val = accuracy_score(target_val, prediccion_val)
            if accuracy_val > best_accuracy:
                best_accuracy = accuracy_val
                best_params = {
                    'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'min_samples_split': min_samples_split
                }

print(f'Mejores parámetros: {best_params}')
print(f'Mejor exactitud en validación: {best_accuracy}')

In [20]:
# Entrenamos el mejor modelo encontrado con el conjunto de prueba
best_model = RandomForestClassifier(
    n_estimators=best_params['n_estimators'], 
    max_depth=best_params['max_depth'], 
    min_samples_split=best_params['min_samples_split'], 
    random_state=54321
)
best_model.fit(features_train, target_train)
prediccion_test = best_model.predict(features_test)
accuracy_test = accuracy_score(target_test, prediccion_test)
print(f'Exactitud del mejor modelo en el conjunto de prueba: {accuracy_test}')

## Conclusion <a id='end'></a>

Se cargaron los datos de comportamiento de los usuarios (users_behavior.csv), y se dividieron en conjuntos de entrenamiento, validación y prueba.
La distribución de los datos fue de 60% para el entrenamiento, 20% para la validación y 20% para la prueba.

Se implementaron y evaluaron tres modelos de aprendizaje automático: DecisionTreeClassifier, RandomForestClassifier y LogisticRegression.

Se realizó una iteración sobre diferentes valores de hiperparámetros (n_estimators, max_depth, min_samples_split) para RandomForestClassifier.

El RandomForestClassifier con los mejores hiperparámetros se evaluó utilizando el conjunto de prueba.

Se concluye que el mejor modelo fue con los siguientes parametros:n_estimators: 200, max_depth: 30 y min_samples_split: 2.

Con una exactitud del modelo de aproximadamente del 80%, superando el umbral mínimo de exactitud requerido del 75%.
